In [1]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("demo").master("spark://spark-master:7077").getOrCreate()
spark = SparkSession.builder \
            .appName("demo") \
            .master("spark://spark-master:7077") \
            .config("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog") \
            .config("spark.sql.catalog.spark_catalog.type","hive") \
            .config("spark.sql.catalog.iceberg","org.apache.iceberg.spark.SparkCatalog") \
            .config("spark.sql.catalog.iceberg.type","hive") \
            .config("spark.sql.catalog.iceberg.uri","thrift://hive-metastore:9083") \
            .config('spark.sql.warehouse.dir', 's3a://lakehouse/') \
            .getOrCreate()

In [11]:
spark.stop()

In [2]:
df = spark.read.format("parquet").load("s3a://lakehouse/bronze/trade/trades.parquet")
df.show(10)
# type(df)

+---+------+-----------+-----------+-----+-----+-----+-----+-------+
| ID|symbol|      value|tradingDate| open| high|  low|close| volume|
+---+------+-----------+-----------+-----+-----+-----+-----+-------+
|  1|   ACB|43975750000| 2013-01-02|16300|17400|16300|17300|2578600|
|  2|   ACB|33790120000| 2013-01-03|17500|17800|16700|17000|1967200|
|  3|   ACB|19893160000| 2013-01-04|16900|17200|16700|17200|1170100|
|  4|   ACB|23335900000| 2013-01-07|17000|17500|17000|17200|1355700|
|  5|   ACB|37493390000| 2013-01-08|17100|17500|16900|17300|2180000|
|  6|   ACB|69459100000| 2013-01-09|17100|18200|17100|18100|3861400|
|  7|   ACB|15857090000| 2013-01-10|18100|18300|17700|18300| 880700|
|  8|   ACB|14849120000| 2013-01-11|19000|19000|18200|18200| 806200|
|  9|   ACB|11046650000| 2013-01-14|18400|18400|17700|18000| 614300|
| 10|   ACB|23515790000| 2013-01-15|18200|18300|17600|18200|1310500|
+---+------+-----------+-----------+-----+-----+-----+-----+-------+
only showing top 10 rows



In [113]:
from pyspark.sql.functions import *
from datetime import datetime, timedelta
from pyspark.sql.types import *
# from pyspark.sql import functions as F
# df = df \
#     .withColumnRenamed("open", "open_price") \
#     .withColumnRenamed("high", "high_price") \
#     .withColumnRenamed("low", "low_price") \
#     .withColumnRenamed("close", "close_price") \
#     .withColumnRenamed("TradingDate", "date") \
#     .drop('ID') \
#     .dropDuplicates(['symbol', 'date'])
# df.count()
# filtered_df = df.filter((col("symbol") == "SSI") & (col("tradingDate") == "2016-09-23"))
# filtered_df.show()
# duplicate_rows = df.groupBy(['symbol', 'date']).count().where(col("count") > 1)
# duplicate_rows.show(30)
start_date = datetime.strptime('2013-01-02', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

# Tạo DataFrame với cột "date"
# date_df 
# date_range = spark.sparkContext.parallelize([(start_date + timedelta(days=x)) for x in range((end_date - start_date).days + 1)])
# date_df = date_range.map(lambda x: (x,)).toDF(['date'])

# date_df = date_df.withColumn("dateKey", year(col("date"))*10000 + month(col("date"))*100 + dayofmonth(col("date"))) \
#     .withColumn("year", year(col("date"))) \
#     .withColumn("quarter", quarter(col("date"))) \
#     .withColumn("month", month(col("date"))) \
#     .withColumn("week", weekofyear(col("date"))) \
#     .withColumn("day", dayofmonth(col("date"))) \
#     .withColumn("day_of_year", dayofyear(col("date"))) \
#     .withColumn("day_name_of_week", date_format(col("date"), "EEEE")) \
#     .withColumn("month_name_of_week", date_format(col("date"), "MMMM")) \
#     .withColumnRenamed("date", "full_date") \
#     .selectExpr(['dateKey', 'full_date', 'year', 'quarter', 'month', 'week', 'day', 'day_of_year', 'day_name_of_week', 'month_name_of_week'])

# date_df.show(10)
start_date

datetime.date(2013, 1, 2)

In [112]:
min_age = df.select(min("tradingDate")).first()[0]
min_age

datetime.date(2013, 1, 2)

In [2]:
# spark.sql(f"CREATE SCHEMA IF NOT EXISTS iceberg.db1")
# spark.sql("CREATE DATABASE IF NOT EXISTS hive_prod.db2 location 's3a://lakehouse/db2.db/';")
# spark.sql("CREATE TABLE IF NOT EXISTS iceberg.db1.stock3s (Symbol string, CompanyName string, Series string, Industry string, ISIN_Code string) USING iceberg")
# spark.sql(f"CREATE TABLE IF NOT EXISTS hive_prod.silver.stocks (Symbol string, CompanyName string, Series string, Industry string, ISIN_Code string) USING iceberg")
# spark.sql("CREATE TABLE IF NOT EXISTS hive_prod.silver.stock2s (Symbol string, CompanyName string, Series string, Industry string, ISIN_Code string) USING iceberg")
# spark.sql(f"CREATE TABLE IF NOT EXISTS hive_prod.silver.stocksss (Symbol varchar(100), CompanyName varchar(100), Series varchar(100), Industry varchar(100), ISIN_Code varchar(100)) USING iceberg")
# spark.sql("SHOW TABLES FROM iceberg.db1").show()
spark.sql("SHOW DATABASES FROM iceberg").show()
# spark.sql("SHOW catalogs").show()

+---------+
|namespace|
+---------+
|  default|
|   silver|
+---------+



In [4]:
df.write.format("iceberg").mode("overwrite").saveAsTable("iceberg.db1.stock")
# df.createOrReplaceTempView("stock2s")

In [4]:
# df.writeTo("iceberg.db1.stock3s").append()

In [ ]:
spark.stop()

In [2]:
cleaned_trades = spark.read.format("iceberg").load("iceberg.silver.cleaned_trades")
dim_date = spark.read.format("iceberg").load("iceberg.silver.dim_date")
dim_company = spark.read.format("iceberg").load("iceberg.silver.dim_company")
fact_table = (
        cleaned_trades
        .join(dim_company, cleaned_trades['symbol'] == dim_company['companyKey'], 'inner')
        .join(dim_date, cleaned_trades['tradingDate'] == dim_date['full_date'], 'inner')
        .select('dateKey', 'companyKey', 'open_price', 'high_price', 'low_price', 'close_price', 'Volume')
    )
fact_table.show(10)

+--------+----------+----------+----------+---------+-----------+-------+
| dateKey|companyKey|open_price|high_price|low_price|close_price| Volume|
+--------+----------+----------+----------+---------+-----------+-------+
|20130228|       CTG|     21100|     21200|    20000|      20000|2846940|
|20130117|       GAS|     43700|     43800|    42000|      42000| 505310|
|20180208|       HDB|     43850|     45850|    43500|      44900|4667920|
|20170123|       NVL|     58200|     58400|    57800|      57800|1087390|
|20140109|       ACB|     15700|     15900|    15700|      15900|  71800|
|20160114|       ACB|     19200|     19200|    18900|      19000| 315858|
|20160301|       ACB|     19700|     19800|    19600|      19600| 140511|
|20161031|       ACB|     19300|     19400|    19000|      19000| 200973|
|20200526|       ACB|     22500|     23500|    22500|      23300|5262900|
|20200909|       ACB|     20600|     20900|    20300|      20900|5019800|
+--------+----------+----------+------

In [68]:
dim_date.show(40)

+--------+-------------------+----+-------+-----+----+---+-----------+----------------+------------------+
| dateKey|          full_date|year|quarter|month|week|day|day_of_year|day_name_of_week|month_name_of_week|
+--------+-------------------+----+-------+-----+----+---+-----------+----------------+------------------+
|20130102|2013-01-02 00:00:00|2013|      1|    1|   1|  2|          2|       Wednesday|           January|
|20130103|2013-01-03 00:00:00|2013|      1|    1|   1|  3|          3|        Thursday|           January|
|20130104|2013-01-04 00:00:00|2013|      1|    1|   1|  4|          4|          Friday|           January|
|20130105|2013-01-05 00:00:00|2013|      1|    1|   1|  5|          5|        Saturday|           January|
|20130106|2013-01-06 00:00:00|2013|      1|    1|   1|  6|          6|          Sunday|           January|
|20130107|2013-01-07 00:00:00|2013|      1|    1|   2|  7|          7|          Monday|           January|
|20130108|2013-01-08 00:00:00|2013|  

In [46]:
from pyspark.sql.functions import *
tdf = (
    fact_table
    .withColumn("day_price_change", col('close_price') - col('open_price'))
    .withColumn("percent_change", bround((col('day_price_change') / col('open_price')) * 100, 2))
    .join(dim_date, 'dateKey', 'inner')
    .join(dim_company, 'companyKey', 'inner')
    .select('companyKey', 'organShortName', 'full_date', 'year', 'day', 'open_price', 'close_price', 'Volume', 'day_price_change', 'percent_change')
)
tdf.show(10)

+----------+--------------------+-------------------+----+---+----------+-----------+-------+----------------+--------------+
|companyKey|      organShortName|          full_date|year|day|open_price|close_price| Volume|day_price_change|percent_change|
+----------+--------------------+-------------------+----+---+----------+-----------+-------+----------------+--------------+
|       CTG|          VietinBank|2013-02-28 00:00:00|2013| 28|     21100|      20000|2846940|           -1100|         -5.21|
|       GAS|              PV Gas|2013-01-17 00:00:00|2013| 17|     43700|      42000| 505310|           -1700|         -3.89|
|       HDB|              HDBank|2018-02-08 00:00:00|2018|  8|     43850|      44900|4667920|            1050|          2.39|
|       NVL|Đầu tư Địa ốc No ...|2017-01-23 00:00:00|2017| 23|     58200|      57800|1087390|            -400|         -0.69|
|       ACB|                 ACB|2014-01-09 00:00:00|2014|  9|     15700|      15900|  71800|             200|        

In [82]:
z = tdf \
    .filter(col('percent_change') > 0) \
    .groupBy('companyKey', 'organShortName', 'full_date') \
    .agg(
        bround(sum('open_price'), 2).alias('open'),
        bround(sum('close_price'), 2).alias('close'),
        bround(sum('day_price_change'), 2).alias('price_change'),
        bround(sum('percent_change'), 2).alias('%_change'),
    ) \
    .orderBy(desc(max('full_date')), desc(max('percent_change')))
z.show()

+----------+--------------------+-------------------+------+------+------------+--------+
|companyKey|      organShortName|          full_date|  open| close|price_change|%_change|
+----------+--------------------+-------------------+------+------+------------+--------+
|       PDR|        BĐS Phát Đạt|2023-06-30 00:00:00| 16450| 16800|         350|    2.13|
|       HPG|            Hòa Phát|2023-06-30 00:00:00| 25700| 26150|         450|    1.75|
|       MWG|    Thế giới di động|2023-06-30 00:00:00| 42700| 43300|         600|    1.41|
|       GVR|Tập đoàn CN Cao s...|2023-06-30 00:00:00| 19150| 19400|         250|    1.31|
|       SSI|     Chứng khoán SSI|2023-06-30 00:00:00| 25500| 25800|         300|    1.18|
|       STB|           Sacombank|2023-06-30 00:00:00| 29550| 29800|         250|    0.85|
|       ACB|                 ACB|2023-06-30 00:00:00| 21900| 22050|         150|    0.68|
|       HDB|              HDBank|2023-06-30 00:00:00| 18500| 18600|         100|    0.54|
|       VJ

In [85]:
x = tdf \
    .filter(col('percent_change') <= 0) \
    .groupBy('companyKey', 'organShortName', 'full_date') \
    .agg(
        bround(sum('open_price'), 2).alias('open'),
        bround(sum('close_price'), 2).alias('close'),
        bround(sum('day_price_change'), 2).alias('price_change'),
        bround(sum('percent_change'), 2).alias('%_change'),
    ) \
    .orderBy(desc(max('full_date')), asc(min('percent_change')))
x.show(50)

+----------+--------------------+-------------------+------+------+------------+--------+
|companyKey|      organShortName|          full_date|  open| close|price_change|%_change|
+----------+--------------------+-------------------+------+------+------------+--------+
|       BID|                BIDV|2023-06-30 00:00:00| 44500| 43350|       -1150|   -2.58|
|       GAS|              PV Gas|2023-06-30 00:00:00| 94600| 93000|       -1600|   -1.69|
|       VIC|            VinGroup|2023-06-30 00:00:00| 51800| 51000|        -800|   -1.54|
|       VNM|            VINAMILK|2023-06-30 00:00:00| 71900| 71000|        -900|   -1.25|
|       POW|Điện lực Dầu khí ...|2023-06-30 00:00:00| 13550| 13400|        -150|   -1.11|
|       VRE|       Vincom Retail|2023-06-30 00:00:00| 27100| 26800|        -300|   -1.11|
|       VCB|         Vietcombank|2023-06-30 00:00:00|101100|100000|       -1100|   -1.09|
|       NVL|Đầu tư Địa ốc No ...|2023-06-30 00:00:00| 15000| 14850|        -150|    -1.0|
|       TC

In [ ]:
data.write.format("iceberg").mode("overwrite").saveAsTable("iceberg.data")

In [ ]:
spark.stop()

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS hive_prod.db")

In [ ]:
spark.sql("CREATE TABLE IF NOT EXISTS hive_prod.db.stock (Symbol string, CompanyName string, Series string, Industry string, ISIN_Code string) USING iceberg")

In [ ]:
# player.writeTo("hive_prod.db.stock").append()
player.writeTo("hive_prod.db.sal").using("iceberg").create()

In [ ]:
icebergdf = spark.read.format("iceberg").load("hive_prod.db.sal")
icebergdf.show(10)

In [ ]:
player = spark.read \
    .format("parquet") \
    .load("s3a://lakehouse/bronze/stock/stocks.parquet")

player.show()

In [ ]:
player.createOrReplaceTempView("player")

In [ ]:
sqlContext = SparkSession(spark)

In [ ]:
t = sqlContext.sql("SELECT * FROM player")

In [ ]:
t.show()

In [ ]:
t.write.format("iceberg").save()

In [ ]:
t = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load("s3a://test/ADANIPORTS.csv")

t.show()

In [ ]:
spark.stop()

In [ ]:
file:///usr/local/spark-3.3.2-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/hadoop-common-3.3.2.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/aws-java-sdk-1.12.367.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/s3-2.18.41.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/aws-java-sdk-bundle-1.11.1026.jar

In [ ]:
spark

In [33]:
from sqlalchemy import create_engine
from sqlalchemy.sql.expression import select, text

engine = create_engine('trino://trino@192.168.59.1:8085/iceberg')
connection = engine.connect()

rows = connection.execute(text("SELECT * FROM db.a")).fetchall()
print(rows)

[('ADANIPORTS', 'Adani Ports and Special Economic Zone Ltd.', 'EQ', 'SERVICES', 'INE742F01042'), ('ASIANPAINT', 'Asian Paints Ltd.', 'EQ', 'CONSUMER GOODS', 'INE021A01026'), ('AXISBANK', 'Axis Bank Ltd.', 'EQ', 'FINANCIAL SERVICES', 'INE238A01034'), ('BPCL', 'Bharat Petroleum Corporation Ltd.', 'EQ', 'ENERGY', 'INE029A01011'), ('BRITANNIA', 'Britannia Industries Ltd.', 'EQ', 'CONSUMER GOODS', 'INE216A01030'), ('CIPLA', 'Cipla Ltd.', 'EQ', 'PHARMA', 'INE059A01026'), ('COALINDIA', 'Coal India Ltd.', 'EQ', 'METALS', 'INE522F01014'), ('DRREDDY', "Dr. Reddy's Laboratories Ltd.", 'EQ', 'PHARMA', 'INE089A01023'), ('EICHERMOT', 'Eicher Motors Ltd.', 'EQ', 'AUTOMOBILE', 'INE066A01013'), ('GAIL', 'GAIL (India) Ltd.', 'EQ', 'ENERGY', 'INE129A01019'), ('GRASIM', 'Grasim Industries Ltd.', 'EQ', 'CEMENT & CEMENT PRODUCTS', 'INE047A01021'), ('BAJAJ-AUTO', 'Bajaj Auto Ltd.', 'EQ', 'AUTOMOBILE', 'INE917I01010'), ('BAJAJFINSV', 'Bajaj Finserv Ltd.', 'EQ', 'FINANCIAL SERVICES', 'INE918I01018'), ('BAJFINA

In [28]:
connection.execute(text("CREATE SCHEMA IF NOT EXISTS iceberg.db with (location = 's3a://lakehouse/')"))

In [34]:
connection.execute(text("CREATE TABLE IF NOT EXISTS iceberg.db.b (Symbol varchar, CompanyName varchar, Series varchar, Industry varchar, ISIN_Code varchar)"))

ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 1:132: mismatched input 'USING'. Expecting: 'COMMENT', 'WITH', <EOF>", query_id=20231102_175215_00163_gh2b3)
[SQL: CREATE TABLE IF NOT EXISTS iceberg.db.b (Symbol varchar, CompanyName varchar, Series varchar, Industry varchar, ISIN_Code varchar) USING iceberg]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df = spark.read.format("iceberg").load("hive_prod.silver.cleaned_stocks")
# df = df.withColumnRenamed("VWAP", "Volume_Weight_Avg_Price") \
#     .withColumn("Date", date_format("Date", "dd-MM-yyyy")) \
#     .drop('ID')
df.show(3)

In [ ]:
# type(df['Open'])
df.describe().toPandas()

In [ ]:
df.distinct().count()

In [ ]:
df.select(['Open', 'High', 'Low', 'Last', 'Close', 'Volume']).describe().toPandas()

In [ ]:
df.groupBy('Symbol').count().show()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
hdfc = df.filter(col('Symbol')=='HDFC')
hdfc.show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.groupBy('Symbol') \
    .agg(min('Date').alias('Start'),
         max('Date').alias('End'),
         
         min('Open').alias('Min_Open'),
         max('Open').alias('Max_Open'),
         avg('Open').alias('Avg_Open'),
         
         min('High').alias('Min_High'),
         max('High').alias('Max_High'),
         avg('High').alias('Avg_High'),
         
         min('Low').alias('Min_Low'),
         max('Low').alias('Max_Low'),
         avg('Low').alias('Avg_Low'),
         
         min('Last').alias('Min_Last'),
         max('Last').alias('Max_Last'),
         avg('Last').alias('Avg_Last'),
         
         min('Close').alias('Min_Close'),
         max('Close').alias('Max_Close'),
         avg('Close').alias('Avg_Close'),
         
         sum('Volume').alias('Sum_Volume'),
        ).toPandas()

In [2]:
df = spark.read.format("iceberg").load("hive_prod.silver.cleaned_trades")
# df = df.withColumnRenamed("VWAP", "Volume_Weight_Avg_Price") \
#     .withColumn("Date", date_format("Date", "dd-MM-yyyy")) \
#     .drop('ID')
df.show(3)

+----------+----------+----------+----------+----------+---------+----------+-----------+-----------------------+--------+----------------+------+------------------+-------------------+
|      Date|    Symbol|Prev_Close|Open_Price|High_Price|Low_Price|Last_Price|Close_Price|Volume_Weight_Avg_Price|  Volume|        Turnover|Trades|Deliverable_Volume|Percent_Deliverable|
+----------+----------+----------+----------+----------+---------+----------+-----------+-----------------------+--------+----------------+------+------------------+-------------------+
|27-11-2007|ADANIPORTS|     440.0|     770.0|    1050.0|    770.0|     959.0|      962.9|                 984.72|27294366|2690000000000000|     0|           9859619|               0.36|
|28-11-2007|ADANIPORTS|     962.9|     984.0|     990.0|    874.0|     885.0|      893.9|                 941.38| 4581338| 431000000000000|     0|           1453278|               0.32|
|29-11-2007|ADANIPORTS|     893.9|     909.0|    914.75|    841.0|    

In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS hive_prod.gold")

DataFrame[]

In [9]:
df.write.format("iceberg").mode("overwrite").saveAsTable("hive_prod.gold.gold1")